In [91]:
from decimal import Decimal
import pandas as pd
import numpy as np
import random 
import math
import csv

In [92]:
def distancia_euclidiana(point1, point2):
    """
    Calculate the Euclidean distance between two points in n-dimensional space.
    
    Arguments:
    point1 (tuple): Coordinates of the first point.
    point2 (tuple): Coordinates of the second point.
    
    Returns:
    float: Euclidean distance between the two points.
    """
    if len(point1) != len(point2):
        raise ValueError("Points must have the same dimensionality")
    
    squared_distance = sum((x - y) ** 2 for x, y in zip(point1, point2))
    distance = math.sqrt(squared_distance)
    return distance

def cria_mapa(x, y, z, block_value = 256):
    
    faixa_central = (x//2)+1
    dimensao_z_por = []
    dimensao_z_perm = []

    ponto_central = (x//2,z//2)
    raio = x/2

    for j in range(0,y):
        plano_xz_por = []
        plano_xz_perm = []
        for i in range(0,x):
            for k in range(0,z):
                ponto = (i,k)
                if distancia_euclidiana(ponto_central,ponto) > raio:
                    plano_xz_por.append(0)
                    plano_xz_perm.append(0)
                else:
                    if j == 0 or j == y-1:
                        plano_xz_por.append(0.01)
                        plano_xz_perm.append(250000)
                    else:
                        perm = block_value * 0.75
                        por = np.log(perm/0.1038)/0.3255/100
                        plano_xz_perm.append(perm)
                        plano_xz_por.append(por)
        dimensao_z_perm.append(plano_xz_perm)
        dimensao_z_por.append(plano_xz_por)
    
    return dimensao_z_perm, dimensao_z_por

def format_cmg2(df,xd,yd,zd):
    '''Transforma os valores do DataFrame no formato do include
    Ex: 5 55 7 =   256 '''
    porosity = []
    y = yd # z = z(dimensão)
    for i in df.columns:
        x = xd # x = x(dimensão)
        z = zd
        for j in df[i]:
            entry = '{0} {1} {2} =   {3:.10f}'.format(x,y,z,j)
            porosity.append(entry)
            z -= 1
            if z == 0: # y = y(dimensão) + 1
                z = zd
                x -= 1
        y -= 1
    porosity_3dmap_cmg = pd.DataFrame(porosity)
    return porosity_3dmap_cmg

In [93]:
perm_11_252_11, por_11_252_11 = cria_mapa(17,352,17)

df_perm = pd.DataFrame(perm_11_252_11)
df_por = pd.DataFrame(por_11_252_11)

ndf_por = df_por.T #transposta para o formato do include do cmg
ndf_perm = df_perm.T 

ndf_por_format = format_cmg2(ndf_por,17,352,17) #formata o DataFrame para o include do cmg
ndf_perm_format = format_cmg2(ndf_perm,17,352,17)


In [94]:
ndf_por_format.to_csv('por_5rand_5x5.inc',sep=' ',header=None, index = False, quoting = csv.QUOTE_NONE, escapechar = ' ')
ndf_perm_format.to_csv('perm_5rand_5x5.inc',sep=' ',header=None, index = False, quoting = csv.QUOTE_NONE, escapechar = ' ')

In [95]:
def cria_vug(geo_vug_xz,x,y,z,block_value = 256):

    tamanho_amostra_y = 5 #cm
    tamanho_amostra_xz = 3.81 #cm

    tam_y = tamanho_amostra_y/y
    tam_xz = tamanho_amostra_xz/x

    geo_vug_y = round((tam_xz/tam_y)*geo_vug_xz)

    perm = block_value*0.75
    por = np.log(perm/0.1038)/0.3255/100

    vug_perm = [[[perm for i in range(geo_vug_xz)] for j in range(geo_vug_y)] for k in range(geo_vug_xz)]
    vug_por = [[[por for i in range(geo_vug_xz)] for j in range(geo_vug_y)] for k in range(geo_vug_xz)]

    return vug_perm, vug_por

In [96]:
def mapear_coordenadas_random_mult(x, y, z, vx, vy, vz, keys, values_perm1, values_por1,n_vugs):
    j = 0
    values_perm = values_perm1
    values_por = values_por1
    while j < n_vugs:
        p = 0
        x_central = random.randint(vx+1, x-vx-1)
        y_central = random.randint(vy+1, y-vy-1)
        z_central = random.randint(vz+1, z-vz-1)

        coordenadas = []

        # Varre todos os pontos ao redor do centro considerando as diferentes distâncias para cada eixo
        for a in range(x_central - vx, x_central + vx + 1):
            for b in range(y_central - vy, y_central + vy + 1):
                for c in range(z_central - vz, z_central + vz + 1):
                    coordenadas.append([a,b,c])

        for i in range(len(coordenadas)):
            if coordenadas[i] in keys:
                indice = keys.index(coordenadas[i])
                if values_perm[indice] != 192: #confirma se é null block ou vug
                    p = 1
                
        if p == 0:
            for i in range(len(coordenadas)):
                if coordenadas[i] in keys:
                    indice = keys.index(coordenadas[i])
                    values_perm[indice] = 2000 # perm de um vug?
                    values_por[indice] = 1

            j += 1
    return values_perm, values_por

In [97]:
def coords_e_valores(include):

    keys = []
    values = []

    file = open(include,'r')

    linhas = file.readlines()

    for linha in linhas:
        lista = linha.split('=')
        coords = lista[0].split('  ')
        coords.remove('')
        keys.append(coords)
        values.append(float(lista[1]))

    keys = [[int(elemento) for elemento in linha] for linha in keys]
            
    return keys,values


In [98]:
def vug_random_mult(vugs,include_perm, include_por, x, y, z, block_value = 256):

    vug_perm, vug_por = cria_vug(4,17,352,17)

    vug_x = round(len(vug_perm)/2)
    vug_y = round((len(vug_perm[0])/2)-0.5)
    vug_z = round(len(vug_perm[0][0])/2)

    keys_perm, values_perm1 = coords_e_valores(include_perm)
    keys_por, values_por1 = coords_e_valores(include_por)
    
    values_perm, values_por = mapear_coordenadas_random_mult(x,y,z,vug_x,vug_y,vug_z,keys_perm,values_perm1,values_por1,vugs)

    # for i in range(len(coordenadas)):
    #     if coordenadas[i] in keys_perm and coordenadas[i] in keys_por:
    #         indice = keys_perm.index(coordenadas[i])
    #         values_perm[indice] = 2000 # perm de um vug?
    #         values_por[indice] = 1.0

    por_file = open(include_por,'w')
    perm_file = open(include_perm,'w')

    for i in range(len(keys_perm)):
        por_file.write(f'{keys_por[i][0]} {keys_por[i][1]} {keys_por[i][2]} = {values_por[i]}\n')
        perm_file.write(f'{keys_perm[i][0]} {keys_perm[i][1]} {keys_perm[i][2]} = {values_perm[i]}\n')
        
    por_file.close()
    perm_file.close()


In [99]:
vug_random_mult(5,'perm_5rand_5x5.inc','por_5rand_5x5.inc',17,352,17)